In [22]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
import matplotlib.pylab as plt
%matplotlib inline

df_all = pd.read_csv("../Data/Federal_Funds_Rate.csv")
df_all['DATE'] = pd.to_datetime(df_all.DATE)
df_all['FEDFUNDS'] = df_all.FEDFUNDS.diff(periods=1)
df_all

In [23]:
df_all = pd.read_csv("../Data/Federal_Funds_Rate.csv")
df_all['DATE'] = pd.to_datetime(df_all.DATE)

In [24]:
###Running the Dickey-Fuller Test to check if data is stationary or not, p-value is second value. P-value above the .05 
##threshold at .061, so FEDFUNDs must be transformed to become stationary
dftest = adfuller(df_all.FEDFUNDS)
print(dftest)

(-2.7757774986763373, 0.06181794891691074, 17, 799, {'1%': -3.4385608049885876, '5%': -2.865164107438179, '10%': -2.5686998068142124}, 926.1124438782902)


In [25]:
df_all['FEDFUNDS'] = df_all.FEDFUNDS.diff(periods=1)

df1 = pd.read_csv('../Data/CPI.csv')
df1['DATE'] = pd.to_datetime(df1.DATE)
df_all = df_all.merge(df1, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')
df_all.CPIAUCSL = df_all.CPIAUCSL.interpolate()

df2 = pd.read_csv('../Data/GDP_Quarterly_Real.csv')
df2['DATE'] = pd.to_datetime(df2.DATE)
df2 = df2.set_index(pd.to_datetime(df2['DATE']), drop=True)
df2.drop('DATE', axis=1, inplace=True)
df2 = df2.resample('MS').interpolate()
df_all = df_all.merge(df2, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')

df3 = pd.read_csv('../Data/Unemployment_Rate.csv')
df3['DATE'] = pd.to_datetime(df3.DATE)
df_all = df_all.merge(df3, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')

df4 = pd.read_csv('../Data/Population_Over55.csv')
df4['DATE'] = pd.to_datetime(df4.DATE)
df_all = df_all.merge(df4, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')

df5 = pd.read_csv('../Data/Population_WorkingAge.csv')
df5['DATE'] = pd.to_datetime(df5.DATE)
df_all = df_all.merge(df5, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')

df6 = pd.read_csv('../Data/M1_MoneySupply.csv')
df6['DATE'] = pd.to_datetime(df6.DATE)
df_all = df_all.merge(df6, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')

df7 = pd.read_csv('../Data/M2_MoneySupply.csv')
df7['DATE'] = pd.to_datetime(df7.DATE)
df_all = df_all.merge(df7, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')
df_all

df8 = pd.read_csv('../Data/M2_velocity_quarterly.csv')
df8['DATE'] = pd.to_datetime(df8.DATE)
df8 = df8.set_index(pd.to_datetime(df8['DATE']), drop=True)
df8.drop('DATE', axis=1, inplace=True)
df8 = df8.resample('MS').interpolate()
df_all = df_all.merge(df8, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')

df9 = pd.read_csv('../Data/Population_Total.csv')
df9['DATE'] = pd.to_datetime(df9.DATE)
df_all = df_all.merge(df9, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')

df10 = pd.read_csv('../Data/Debt_to_GDP_Quarterly.csv')
df10['DATE'] = pd.to_datetime(df10.DATE)
df10 = df10.set_index(pd.to_datetime(df10['DATE']), drop=True)
df10.drop('DATE', axis=1, inplace=True)
df10 = df10.resample('MS').interpolate()
df_all = df_all.merge(df10, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')

df11 = pd.read_csv('../Data/Federal_Debt_Quarterly.csv')
df11['DATE'] = pd.to_datetime(df11.DATE)
df11 = df11.set_index(pd.to_datetime(df11['DATE']), drop=True)
df11.drop('DATE', axis=1, inplace=True)
df11 = df11.resample('MS').interpolate()
df_all = df_all.merge(df11, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')

In [26]:
df12 = pd.read_csv('../Data/USDJPY_monthly.csv')
df12['DATE'] = pd.to_datetime(df12.DATE)
df_all = df_all.merge(df12, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')

df13 = pd.read_csv('../Data/Core_CPI.csv')
df13['DATE'] = pd.to_datetime(df13.DATE)
df_all = df_all.merge(df13, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')

df14 = pd.read_csv('../Data/Inital_Claims_4WeekMA.csv')
df14['DATE'] = pd.to_datetime(df14.DATE)
df14 = df14.set_index(pd.to_datetime(df14['DATE']), drop=True)
df14.drop('DATE', axis=1, inplace=True)
df14 = df14.resample('MS').mean()
df_all = df_all.merge(df14, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')

df15 = pd.read_csv('../Data/Continuing_Claims_4WeekMA.csv')
df15['DATE'] = pd.to_datetime(df15.DATE)
df15 = df15.set_index(pd.to_datetime(df15['DATE']), drop=True)
df15.drop('DATE', axis=1, inplace=True)
df15 = df15.resample('MS').mean()
df_all = df_all.merge(df15, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')

df16 = pd.read_csv('../Data/Mortgage_Rate_Weekly.csv')
df16['DATE'] = pd.to_datetime(df16.DATE)
df16 = df16.set_index(pd.to_datetime(df16['DATE']), drop=True)
df16.drop('DATE', axis=1, inplace=True)
df16 = df16.resample('MS').mean()
df_all = df_all.merge(df16, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')

df17 = pd.read_csv('../Data/Capacity_Utilization_Ind.csv')
df17['DATE'] = pd.to_datetime(df17.DATE)
df_all = df_all.merge(df17, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')

df18 = pd.read_csv('../Data/Total_Nonfarmpayrolls.csv')
df18['DATE'] = pd.to_datetime(df18.DATE)
df_all = df_all.merge(df18, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')

df19 = pd.read_csv('../Data/PPI_AllCommods.csv')
df19['DATE'] = pd.to_datetime(df19.DATE)
df_all = df_all.merge(df19, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')

df20 = pd.read_csv('../Data/Total_ConsumerCredit.csv')
df20['DATE'] = pd.to_datetime(df20.DATE)
df_all = df_all.merge(df20, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')

df21 = pd.read_csv('../Data/SPOT_WTI.csv')
df21['DATE'] = pd.to_datetime(df21.DATE)
df_all = df_all.merge(df21, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')

df22 = pd.read_csv('../Data/Motor_VehicleLoans.csv')
df22['DATE'] = pd.to_datetime(df22.DATE)
df22 = df22.set_index(pd.to_datetime(df22['DATE']), drop=True)
df22.drop('DATE', axis=1, inplace=True)
df22 = df22.resample('MS').interpolate()
df_all = df_all.merge(df22, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')

df23 = pd.read_csv('../Data/Federal_InterestPayments.csv')
df23['DATE'] = pd.to_datetime(df23.DATE)
df23 = df23.set_index(pd.to_datetime(df23['DATE']), drop=True)
df23.drop('DATE', axis=1, inplace=True)
df23 = df23.resample('MS').interpolate()
df_all = df_all.merge(df23, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')

df24 = pd.read_csv('../Data/Federal_TaxReceipts.csv')
df24['DATE'] = pd.to_datetime(df24.DATE)
df24 = df24.set_index(pd.to_datetime(df24['DATE']), drop=True)
df24.drop('DATE', axis=1, inplace=True)
df24 = df24.resample('MS').interpolate()
df_all = df_all.merge(df24, on='DATE', how='outer')
df_all = df_all.sort_values('DATE')
df_all

,DATE,FEDFUNDS,CPIAUCSL,GDPC1,UNRATE,LNU00024230,LNU00000060,M1SL,M2SL,M2V,...,CC4WSA,MORTGAGE30US,TCU,PAYEMS,PPIACO,TOTALSL,WTISPLC,MVLOAS,A091RC1Q027SBEA,W006RC1Q027SBEA
0,1913-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,12.100,NaN,NaN,NaN,NaN,NaN
1,1913-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,12.000,NaN,NaN,NaN,NaN,NaN
2,1913-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,12.000,NaN,NaN,NaN,NaN,NaN
3,1913-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,12.000,NaN,NaN,NaN,NaN,NaN
4,1913-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,11.900,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311,2022-04-01,0.13,288.663,19681.682,3.6,98528.0,127126.0,20615.3,21655.5,1.147,...,1449500.0,4.9825,80.4220,151224.0,265.385,4563.48947,101.78,1365.13043,599.097,NaN
1312,2022-05-01,0.44,291.474,NaN,3.6,98646.0,127115.0,20620.7,21684.4,NaN,...,1338312.5,5.2300,80.3335,151610.0,273.270,4587.28211,109.55,NaN,NaN,NaN
1313,2022-06-01,0.44,295.328,NaN,3.6,98779.0,127121.0,20545.4,21667.5,NaN,...,1320937.5,5.5220,80.0490,152008.0,282.360,4627.43653,114.84,NaN,NaN,NaN
1314,2022-07-01,0.47,295.328,NaN,3.5,98908.0,127149.0,NaN,NaN,NaN,...,1358187.5,5.4125,NaN,152536.0,NaN,NaN,101.62,NaN,NaN,NaN


In [27]:
df_all.set_index('DATE', inplace=True)

In [28]:
##Forward filling the 3 variables that most recent data points end 1/1/2022 to forward 4/1/2022
df_all.GFDEGDQ188S = df_all.GFDEGDQ188S.ffill()
df_all.GFDEBTN = df_all.GFDEBTN.ffill()
df_all.W006RC1Q027SBEA = df_all.W006RC1Q027SBEA.ffill()

In [29]:
##Displays all columns
##pd.set_option('display.max_columns', None)
##resets max columns
##pd.reset_option("^display")

In [31]:
df_all['Fed_Int_Over_Receipts'] = (df_all['A091RC1Q027SBEA'] / df_all['W006RC1Q027SBEA'])
df_all

,FEDFUNDS,CPIAUCSL,GDPC1,UNRATE,LNU00024230,LNU00000060,M1SL,M2SL,M2V,POPTHM,...,MORTGAGE30US,TCU,PAYEMS,PPIACO,TOTALSL,WTISPLC,MVLOAS,A091RC1Q027SBEA,W006RC1Q027SBEA,Fed_Int_Over_Receipts
DATE,,,,,,,,,,,,,,,,,,,,,
1913-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,12.100,NaN,NaN,NaN,NaN,NaN,NaN
1913-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,12.000,NaN,NaN,NaN,NaN,NaN,NaN
1913-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,12.000,NaN,NaN,NaN,NaN,NaN,NaN
1913-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,12.000,NaN,NaN,NaN,NaN,NaN,NaN
1913-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,11.900,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-01,0.13,288.663,19681.682,3.6,98528.0,127126.0,20615.3,21655.5,1.147,332863.0,...,4.9825,80.4220,151224.0,265.385,4563.48947,101.78,1365.13043,599.097,2913.013,0.205662
2022-05-01,0.44,291.474,NaN,3.6,98646.0,127115.0,20620.7,21684.4,NaN,332928.0,...,5.2300,80.3335,151610.0,273.270,4587.28211,109.55,NaN,NaN,2913.013,NaN
2022-06-01,0.44,295.328,NaN,3.6,98779.0,127121.0,20545.4,21667.5,NaN,333028.0,...,5.5220,80.0490,152008.0,282.360,4627.43653,114.84,NaN,NaN,2913.013,NaN


Chose 4/1/2022 data end date because it represents the best date. Encompasses the beginning of the rate hiking cycle (shows in march monthly fed funds rate increase) while also limiting the ffills to only three variables. If chose 7/1/2022 end date, would need to forward fill 12 independent variables. 

In [32]:
df_all = df_all['1971-07-01':'2022-04-01']

In [33]:
##final data set
df_all

,FEDFUNDS,CPIAUCSL,GDPC1,UNRATE,LNU00024230,LNU00000060,M1SL,M2SL,M2V,POPTHM,...,MORTGAGE30US,TCU,PAYEMS,PPIACO,TOTALSL,WTISPLC,MVLOAS,A091RC1Q027SBEA,W006RC1Q027SBEA,Fed_Int_Over_Receipts
DATE,,,,,,,,,,,,,,,,,,,,,
1971-07-01,0.40,40.600,5142.422000,6.0,38011.0,70789.0,224.9,679.6,1.717000,207771.0,...,7.6040,79.2256,71315.0,38.300,139.90802,3.56,39.412000,35.408000,135.276000,0.261746
1971-08-01,0.26,40.700,5146.463667,6.1,38070.0,70938.0,225.6,685.5,1.708000,207998.0,...,7.6975,78.5675,71373.0,38.500,141.25209,3.56,39.781000,35.616000,136.066667,0.261754
1971-09-01,-0.02,40.800,5150.505333,6.0,38142.0,71087.0,226.5,692.5,1.699000,208230.0,...,7.6875,79.6440,71614.0,38.300,142.76657,3.56,40.150000,35.824000,136.857333,0.261762
1971-10-01,-0.35,40.900,5154.547000,5.8,38214.0,71238.0,227.2,698.4,1.690000,208450.0,...,7.6280,80.0392,71642.0,38.300,143.92676,3.56,40.519000,36.032000,137.648000,0.261769
1971-11-01,-0.29,41.000,5186.143667,6.0,38273.0,71362.0,227.8,704.6,1.692000,208648.0,...,7.5500,80.1781,71847.0,38.300,145.63484,3.56,41.236667,36.197333,141.976000,0.254954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-01,0.00,280.126,19754.042000,3.9,98741.0,126205.0,20430.7,21490.0,1.124333,332640.0,...,3.0980,78.7416,149240.0,241.338,4430.82582,71.71,1326.600173,554.535667,2814.255000,0.197045
2022-01-01,0.00,281.933,19727.918000,4.0,98159.0,127164.0,20585.4,21649.6,1.124000,332684.0,...,3.4450,78.9421,149744.0,246.453,4446.72147,83.22,1332.846310,561.075000,2913.013000,0.192610
2022-02-01,0.00,284.182,19712.506000,3.8,98255.0,127180.0,20661.3,21708.6,1.131667,332750.0,...,3.7625,79.4912,150458.0,252.660,4481.40108,91.64,1343.607683,573.749000,2913.013000,0.196961


In [34]:
##zero nulls within the data set 
df_all.isna().sum()

FEDFUNDS                 0
CPIAUCSL                 0
GDPC1                    0
UNRATE                   0
LNU00024230              0
LNU00000060              0
M1SL                     0
M2SL                     0
M2V                      0
POPTHM                   0
GFDEGDQ188S              0
GFDEBTN                  0
EXJPUS                   0
CPILFESL                 0
IC4WSA                   0
CC4WSA                   0
MORTGAGE30US             0
TCU                      0
PAYEMS                   0
PPIACO                   0
TOTALSL                  0
WTISPLC                  0
MVLOAS                   0
A091RC1Q027SBEA          0
W006RC1Q027SBEA          0
Fed_Int_Over_Receipts    0
dtype: int64

As we can see above, after testing our target variable and 3 key independent variables we can confirm that the data is not stationary, and needs to be transformed. (Does not reach the .05 p-value threshold)

In [ ]:
##for column in df_all_diff:
  ###  print(adfuller(df_all_diff[column]))

P-values have moved lower than the .05 p-value threshold. Most of DateFrame df_all_diff is stationary. Need to further transform column 5, 6, 10, 12, 14, 23 (LNU00024230, LNU00000060, POPTHM, GFDEBTN, CPILFESL, MVLOAS). Population, federal debt, Core CPI, motor vehicle loans